# Analyse de la qualité des données du fichier produits 
Cet extract CSV provient du fournisseur

## Résumé des anomalies observées
- 8% des produits n'ont pas de référence (null)
--> problème de complétude : l'incrément ne s'est pas fait
- 10% des produits ont un prix négatif 
<br>--> erreur de saisie (problème saisie manuelle)
- 100% des produits ont plus de chiffres après la virgule
  <br> --> erreur de cohérence (problème de conversion d'une devise étrangère)
- 100% des produits ont une description commençant par 'Produit' et réplique ce champs
<br> + problème de cohérence : le dernier mot ne correspond pas à une description
<br>--> erreur de contamination/redondance (problème de mapping)
<br> ces données ne peuvent pas être corrigées, elles sont inutilisables
<br>
<br>Pas de doublon (aucune ligne dupliquée)
<br>La colonne 'date_mise_a_jour' semble avoir un format cohérent et valide 

## Proposition d'une correction
- les références manquantes peuvent être retrouvées par interpolation (n-1 / n+1)
<br> --> 100% de correction validée
-  les prix négatifs semblent cohérents avec le produit, on peut inverser les signes
<br> --> 100% de correction validée
-  les descriptions ne sont pas exploitables (1 seul mot vide de sens)
<br> --> la colonne est simplement supprimée

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_products = pd.read_csv('../data/raw/Catalogue_Produits.csv')

In [3]:
print("AUDIT DATAFRAME MARKETPLACE")

display(df_products.head())
display(df_products.info())

print("\n---Taux de valeurs manquantes (%)---")
missing_percentage = df_products.isnull().sum() / len(df_products) * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

print("\n---Vérification des doublons---")
print("Nombre total de doublons :", df_products.duplicated().sum())

print("\n--- Vérification des doublons sur ref_produits (autres que null) ---")
df_non_null = df_products[df_products['ref_produit'].notna()]
nb_doublons = df_non_null['ref_produit'].duplicated().sum()
print(f"Nombre de références produits en doublon : {nb_doublons}")
if nb_doublons > 0:
    # Afficher les références en doublon
    refs_dupliquees = df_products[df_products['ref_produit'].duplicated(keep=False)]
    print(f"\nNombre total de lignes concernées : {len(refs_dupliquees)}")

    # Afficher quelques exemples de lignes dupliquées
    print("\nExemples de lignes avec références dupliquées :")
    display(refs_dupliquees.sort_values('ref_produit').head(10))


print("\n---Vérification des prix négatifs---")
print("Nombre total de prix négatifs :",(df_products['prix_fournisseur']<0).sum())

lignes_prix_negatifs = df_products[df_products['prix_fournisseur'] < 0]
print("\nExemples de lignes avec des prix négatifs :")
display(lignes_prix_negatifs.head(8))

print("\n--- Analyse de la précision des prix ---")
prix_anormaux = df_products['prix_fournisseur'].apply(lambda x: len(str(x).split('.')[-1]) > 2 if '.' in str(x) else False)
print(f"Nombre de prix avec >2 décimales : {prix_anormaux.sum()}")
print(f"{prix_anormaux.sum()/len(df_products) * 100}% des produits ont plus de 2 décimales")
print("Il doit donc y avoir un problème de conversion des prix.")

print("\n--- Analyse du mot 'produit' dans la description ---")
# Vérifier si la description commence exactement par "Produit " suivi du nom du produit
def verifie_format_produit(row):
    if pd.isna(row['description']) or pd.isna(row['produit']):
        return False

    format_attendu = f"Produit {row['produit']}"
    return row['description'].startswith(format_attendu)

mask_format_produit = df_products.apply(verifie_format_produit, axis=1)

nb_avec_format = mask_format_produit.sum()

print(f"Nombre de lignes où la description correspond au nom du produit : {nb_avec_format}")
# print(f"Pourcentage : {(nb_avec_format / len(df_products)) * 100:.2f}%")
print(f"{nb_avec_format/len(df_products) * 100:.2f}% des descriptions commencent par 'Produit' et répliquent ce champs")
print("Il doit donc y avoir un problème dans la configuration de l'export côté fournisseur.")


print("\n--- Analyse de Cohérence de la Colonne Date ---") # Code reprise de Romaric
try:
    # Tenter la conversion en utilisant 'coerce' pour mettre les dates invalides à NaT (Not a Time)
    df_products['date_temp'] = pd.to_datetime(df_products['date_mise_a_jour'], errors='coerce')

    # Compter le nombre de dates qui n'ont pas pu être converties (erreurs de format)
    invalid_dates_count = df_products['date_temp'].isna().sum() - df_products['date_mise_a_jour'].isna().sum()

    if invalid_dates_count > 0:
        print(f"{invalid_dates_count} entrées de la colonne 'date' n'ont pas un format de date valide.")
        # Afficher les valeurs d'origine qui ont échoué la conversion
        invalid_dates = df_products[df_products['date_temp'].isna() & df_products['date_mise_a_jour'].notna()]['date_mise_a_jour'].unique()
        print(f"Exemples de formats invalides : {invalid_dates[:5]}")
    else:
        print("La colonne 'date' semble avoir un format cohérent et valide pour la conversion en datetime.")

    df_products.drop(columns=['date_temp'], inplace=True) # Supprimer la colonne temporaire

except Exception as e:
    print(f"Une erreur inattendue est survenue lors de l'analyse de la date : {e}")

print("\n--- Analyse de l'ancienneté des mises à jour ---")

# Convertir la colonne date_mise_a_jour en datetime
df_products['date_mise_a_jour_dt'] = pd.to_datetime(df_products['date_mise_a_jour'], errors='coerce')

# Date actuelle
date_actuelle = pd.Timestamp.now()

# Date limite (2 ans en arrière)
date_limite = date_actuelle - pd.DateOffset(years=2)

# Calculer l'ancienneté et filtrer les produits >2 ans
produits_anciens = df_products['date_mise_a_jour_dt'] < date_limite
nb_produits_anciens = produits_anciens.sum()

# Calculer le pourcentage (en excluant les valeurs NaT)
nb_produits_valides = df_products['date_mise_a_jour_dt'].notna().sum()
pourcentage_anciens = (nb_produits_anciens / nb_produits_valides) * 100

print(f"Nombre de produits avec date de mise à jour > 2 ans : {nb_produits_anciens}")
print(f"Pourcentage : {pourcentage_anciens:.2f}%")
print(f"Date limite considérée : {date_limite.strftime('%Y-%m-%d')}")

# Nettoyer la colonne temporaire
df_products.drop(columns=['date_mise_a_jour_dt'], inplace=True)

AUDIT DATAFRAME MARKETPLACE


,produit,ref_produit,description,prix_fournisseur,date_mise_a_jour
0,Écran 24 pouces,REF-1001,Produit Écran 24 pouces figure,66.556477,2025-07-26
1,Webcam HD,REF-1002,Produit Webcam HD aussi,121.868920,2024-09-23
2,Casque Bluetooth,REF-1003,Produit Casque Bluetooth choisir,59.158904,2023-02-22
3,Enceinte Bluetooth,NaN,Produit Enceinte Bluetooth chercher,-21.284130,2022-12-03
4,Chargeur rapide,REF-1005,Produit Chargeur rapide sien,97.507855,2023-02-08


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   produit           100 non-null    object 
 1   ref_produit       92 non-null     object 
 2   description       100 non-null    object 
 3   prix_fournisseur  100 non-null    float64
 4   date_mise_a_jour  100 non-null    object 
dtypes: float64(1), object(4)
memory usage: 4.0+ KB


None


---Taux de valeurs manquantes (%)---
ref_produit    8.0
dtype: float64

---Vérification des doublons---
Nombre total de doublons : 0

--- Vérification des doublons sur ref_produits (autres que null) ---
Nombre de références produits en doublon : 0

---Vérification des prix négatifs---
Nombre total de prix négatifs : 10

Exemples de lignes avec des prix négatifs :


,produit,ref_produit,description,prix_fournisseur,date_mise_a_jour
3,Enceinte Bluetooth,NaN,Produit Enceinte Bluetooth chercher,-21.284130,2022-12-03
5,Disque SSD,REF-1006,Produit Disque SSD esprit,-133.209937,2023-12-29
15,Enceinte Bluetooth,NaN,Produit Enceinte Bluetooth avant,-70.506313,2024-08-01
29,Souris,REF-1030,Produit Souris préparer,-92.466814,2023-05-30
42,Webcam HD,REF-1043,Produit Webcam HD bonheur,-12.589137,2024-06-19
43,Clavier,REF-1044,Produit Clavier effet,-57.626919,2025-08-31
45,Écran 24 pouces,REF-1046,Produit Écran 24 pouces point,-13.720640,2023-11-14
46,Enceinte Bluetooth,REF-1047,Produit Enceinte Bluetooth question,-143.855666,2023-11-20



--- Analyse de la précision des prix ---
Nombre de prix avec >2 décimales : 100
100.0% des produits ont plus de 2 décimales
Il doit donc y avoir un problème de conversion des prix.

--- Analyse du mot 'produit' dans la description ---
Nombre de lignes où la description correspond au nom du produit : 100
100.00% des descriptions commencent par 'Produit' et répliquent ce champs
Il doit donc y avoir un problème dans la configuration de l'export côté fournisseur.

--- Analyse de Cohérence de la Colonne Date ---
La colonne 'date' semble avoir un format cohérent et valide pour la conversion en datetime.

--- Analyse de l'ancienneté des mises à jour ---
Nombre de produits avec date de mise à jour > 2 ans : 37
Pourcentage : 37.00%
Date limite considérée : 2023-12-02


## Correction des incréments manquants pour ref_produit

In [4]:
print("\n--- CORRECTION DES RÉFÉRENCES MANQUANTES PAR ITERATION et INTERPOLATION ---")

# copie du dataframe original
df_products_index_corrected = df_products.copy()

# Fonction pour extraire le numéro d'une référence
def extraire_numero(ref):
    """Extrait le numéro d'une référence type REF-1006"""
    if pd.isna(ref):
        return None
    try:
        return int(str(ref).split('-')[-1])
    except:
        return None

# Fonction pour construire une référence
def construire_ref(numero):
    """Construit une référence type REF-1006"""
    return f"REF-{numero}"

# Itérer sur chaque ligne
nb_corriges = 0

for i in range(len(df_products_index_corrected)):
    # Vérifier si la référence actuelle est manquante
    if pd.isna(df_products_index_corrected.loc[i, 'ref_produit']):

        # Récupérer le numéro de la ligne précédente (n-1)
        if i > 0:
            num_precedent = extraire_numero(df_products_index_corrected.loc[i-1, 'ref_produit'])
        else:
            num_precedent = None

        # Récupérer le numéro de la ligne suivante (n+1)
        if i < len(df_products_index_corrected) - 1:
            num_suivant = extraire_numero(df_products_index_corrected.loc[i+1, 'ref_produit'])
        else:
            num_suivant = None

        # Vérifier si l'incrément est de +2
        if num_precedent is not None and num_suivant is not None:
            if num_suivant == num_precedent + 2:
                # Calculer la valeur manquante (n-1 + 1)
                num_manquant = num_precedent + 1

                # Inscrire la référence corrigée
                df_products_index_corrected.loc[i, 'ref_produit'] = construire_ref(num_manquant)
                nb_corriges += 1

print(f"✅ {nb_corriges} références corrigées")
print(f"Références encore nulles : {df_products_index_corrected['ref_produit'].isna().sum()}")



--- CORRECTION DES RÉFÉRENCES MANQUANTES PAR ITERATION et INTERPOLATION ---
✅ 8 références corrigées
Références encore nulles : 0


In [5]:
print("\n--- SAUVEGARDE DU FICHIER AVEC INDEX CORRIGÉS ---")

# Sauvegarder dans le dossier processed
df_products_index_corrected.to_csv('../data/processed/Catalogue_Produits_corrected.csv', index=False)


--- SAUVEGARDE DU FICHIER AVEC INDEX CORRIGÉS ---


## Inversion des prix négatifs

In [14]:
print("\n--- INVERSION DES PRIX NÉGATIFS ---")

df_products_price_corrected = pd.read_csv('../data/raw/Catalogue_Produits.csv')
mask_prix_negatifs = df_products_price_corrected['prix_fournisseur'] < 0
nb_negatifs_avant = mask_prix_negatifs.sum()

# Exporter TOUTES les lignes avec prix négatifs AVANT correction
df_products_price_negative = df_products_price_corrected[mask_prix_negatifs].copy()
display(df_products_price_negative.head())
print("\n--- SAUVEGARDE DU FICHIER AVEC LES PRIX NEGATIFS ---")
df_products_price_negative.to_csv('../data/processed/Catalogue_Produits_negatifs.csv', index=False)

# Inverser uniquement les prix négatifs dans la copie
df_products_price_corrected.loc[mask_prix_negatifs, 'prix_fournisseur'] = df_products_price_corrected.loc[mask_prix_negatifs, 'prix_fournisseur'].abs()


nb_negatifs_apres = (df_products_price_corrected['prix_fournisseur'] < 0).sum()

print(f"✅ {nb_negatifs_avant} prix inversés")
print(f"Prix négatifs restants : {nb_negatifs_apres}")


--- INVERSION DES PRIX NÉGATIFS ---


,produit,ref_produit,description,prix_fournisseur,date_mise_a_jour
3,Enceinte Bluetooth,NaN,Produit Enceinte Bluetooth chercher,-21.284130,2022-12-03
5,Disque SSD,REF-1006,Produit Disque SSD esprit,-133.209937,2023-12-29
15,Enceinte Bluetooth,NaN,Produit Enceinte Bluetooth avant,-70.506313,2024-08-01
29,Souris,REF-1030,Produit Souris préparer,-92.466814,2023-05-30
42,Webcam HD,REF-1043,Produit Webcam HD bonheur,-12.589137,2024-06-19



--- SAUVEGARDE DU FICHIER AVEC LES PRIX NEGATIFS ---
✅ 10 prix inversés
Prix négatifs restants : 0


In [8]:
print("\n--- SAUVEGARDE DU FICHIER AVEC PRIX CORRIGÉS ---")

# Sauvegarder dans le dossier processed
df_products_price_corrected.to_csv('../data/processed/Catalogue_Produits_corrected.csv', index=False)


--- SAUVEGARDE DU FICHIER AVEC PRIX CORRIGÉS ---


## Vérification de la pertinence d'une modification
s'il ne reste toujours qu'un seul mot après suppression de la réplication, alors 
le champs description peut être supprimé car son contenu est inexploitable

In [ ]:
print("\n--- Analyse du contenu restant après suppression ---")

def analyser_reste_description(row):
    """Extrait ce qui reste après 'Produit [nom_produit]' """
    if pd.isna(row['description']) or pd.isna(row['produit']):
        return None

    format_attendu = f"Produit {row['produit']}"

    if row['description'].startswith(format_attendu):
        # Extraire ce qui reste après la partie dupliquée
        reste = row['description'][len(format_attendu):].strip()
        return reste if reste else None

    return None

# Appliquer sur les lignes avec duplication
df_products['reste_description'] = df_products.apply(analyser_reste_description, axis=1)

# Analyser les restes non vides
restes_non_vides = df_products[df_products['reste_description'].notna()]

if len(restes_non_vides) > 0:
    print(f"Lignes avec contenu restant après 'Produit [nom]' : {len(restes_non_vides)}")

    # Compter le nombre de mots dans le reste
    restes_non_vides['nb_mots_reste'] = restes_non_vides['reste_description'].str.split().str.len()

    print(f"\nDistribution du nombre de mots restants :")
    print(restes_non_vides['nb_mots_reste'].value_counts().sort_index())

    # Vérifier si c'est toujours 1 mot
    nb_un_seul_mot = (restes_non_vides['nb_mots_reste'] == 1).sum()
    pct_un_seul_mot = (nb_un_seul_mot / len(restes_non_vides)) * 100

    print(f"\n✅ Lignes avec exactement 1 mot restant : {nb_un_seul_mot} ({pct_un_seul_mot:.1f}%)")

    if pct_un_seul_mot == 100:
        print("→ CONFIRMATION : Le reste est toujours constitué d'un seul mot")
        print("→ Structure identifiée : 'Produit [nom_produit] [mot_unique]'")
    else:
        print("✅ Aucun contenu supplémentaire après 'Produit [nom_produit]'")
        print("→ Les descriptions sont exactement : 'Produit [nom_produit]'")

# Nettoyer les colonnes temporaires
df_products.drop(columns=['reste_description'], inplace=True, errors='ignore')
if 'nb_mots_reste' in df_products.columns:
    df_products.drop(columns=['nb_mots_reste'], inplace=True)


--- Analyse du contenu restant après suppression ---
Lignes avec contenu restant après 'Produit [nom]' : 100

Distribution du nombre de mots restants :
nb_mots_reste
1    100
Name: count, dtype: int64

✅ Lignes avec exactement 1 mot restant : 100 (100.0%)
→ CONFIRMATION : Le reste est toujours constitué d'un seul mot
→ Structure identifiée : 'Produit [nom_produit] [mot_unique]'


In [ ]:
print("\n--- SUPPRESSION DE LA COLONNE 'description' ---")

df_products_corrected = pd.read_csv('../data/processed/Catalogue_Produits_corrected.csv')

# Supprimer la colonne description
df_products_final = df_products_corrected.drop(columns=['description'])

# Sauvegarder dans un fichier CSV
df_products_final.to_csv('../data/processed/Catalogue_Produits_corrected.csv', index=False)


--- SUPPRESSION DE LA COLONNE 'description' ---
